### Run the app by running the below cell
### Controls:
Right-click (on stroke): turn on edit mode

Space: Toggle control point editor

Delete: Delete the currently edited stroke

Left-click (in the frame): contextual to move control points or stroke box position

In [ ]:
from tkinter import *
import numpy as np
import re #regex
#back end libraries for loading/saving/modifying our data
import StrokeFit as sf
from FitDef import FitData
import FitDef as fd
from StrokeDef import StrokeType #needed for loading object properly
import StrokeDef as sd
import FormalImage as fi
import BatchProcess as bp
import XMLMaker
import cv2 #allows resizing images
from PIL import Image, ImageTk #allows settting canvas image from array
import bezier
import colorsys
import platform #used to check if on mac or windows

#helper function for coloring
def getColor(h,s,v): #get hex code from hsv
    rgb = tuple((np.array(colorsys.hsv_to_rgb(h,s,v))*255).astype(int)) #very ugly, goes from 0-1 rgb, to 255 valued
    return "#" + '%02x%02x%02x' % rgb

def getColors(num, s=.9, v=.5): #will evenly spread colors across hues
    hues = np.linspace(0,1,num+1)[:-1] #ignore the last since hue is cyclic
    colors = []
    for hue in hues:
        colors.append(getColor(hue,s,v))
    return colors

def betweenDist(coord, minV, maxV): #helper function for figuring out click distance between two lines
    if(minV > maxV):
        return betweenDist(coord, maxV, minV) #minV and maxV are in wrong order, swap them
    if(coord < minV):
        return minV - coord
    elif(coord > maxV):
        return maxV - coord
    else: #between
        return 0

#classes for each of their frames and corresponding events
class CharFrame():
    def __init__(self, app):
        self.frame = Frame(app.topFrame, bg=app.lightBlue)
        self.frame.pack(side='left',  fill='both', padx=app.padding,  pady=app.padding)
        #var for entries
        self.code = StringVar(value="4f4d")
        self.char = StringVar(value="\u4f4d")
        #content
        self.label = Label(self.frame, bg=app.lightBlue, text="Symbol:", font= ('Arial 22'), padx=15)
        self.label.pack(side='left')
        self.codeEntry = Entry(self.frame, text="", font= ('Arial 22'), width=8, textvariable=self.code)
        self.codeEntry.pack(side='left', padx=25)
        self.charEntry = Entry(self.frame, text="", font= ('Arial 22'), width=4, textvariable=self.char)
        self.charEntry.pack(side='left', padx=15)

        
class StrokeItem():
    def __init__(self, sf, currNum, bgColor, allowDelete=True): #sf be the StrokeFrame class, some stroke may not be deleted if they are auto generated.
        self.strokeChoice = StringVar(value=sf.strokeOptions[0]) #default option
        self.groupChoice = StringVar(value=sf.groupOptions[0])
        
        self.strokeEvent = None #these are the trace function for the corresponding variable change
        self.groupEvent = None
        
        self.editColor = getColor(.6, .5, .8) #otherwise white
        self.editMode = False
        
        self.frame = Frame(sf.scrollFrame, bg=app.offWhite, width=375, height=35, highlightthickness=0, 
                           highlightcolor=self.editColor, highlightbackground=getColor(.6, .7, .9))
        self.frame.pack_propagate(0)
        self.frame.pack(side='top', padx=2, pady=2)
        #content
        self.nameLabel = Label(self.frame, text="#" + str(currNum), bg=app.offWhite, font=('Arial 10'), fg="white")
        self.nameLabel.pack(side="left", padx=[0,3])
        #self.nameLabel.grid(sticky="w",row=0, column=0, columnspan=2)
        if(allowDelete):
            self.deleteButton = Button(self.frame, text="X", font=('Arial 12'), bg="#F34848")
            self.deleteButton.pack(side="right", anchor="nw")
            #self.deleteButton.grid(sticky="w", row=0, column=4)
        self.typeLabel = Label(self.frame, text="Type:", bg=app.offWhite, font=('Arial 10'), fg="white")
        self.typeLabel.pack(side="left")
        #self.typeLabel.grid(row=1, column=0)
        self.typeOptions = OptionMenu(self.frame, self.strokeChoice, *sf.strokeOptions)
        self.typeOptions.config()
        self.typeOptions.pack(side="left")
        #self.typeOptions.grid(row=1, column=1)
        self.groupLabel = Label(self.frame, text="Group:", bg=app.offWhite, font=('Arial 10'), fg="white")
        self.groupLabel.pack(side="left")
        #self.groupLabel.grid(row=2, column=0)
        self.groupOptions = OptionMenu(self.frame, self.groupChoice, *sf.groupOptions)
        self.groupOptions.config()
        self.groupOptions.pack(side="left")
        #self.groupOptions.grid(row=2, column=1)
        self.detectButton = Button(self.frame, text="Detect", font=('Arial 10'))
        #self.detectButton.pack(side="left", padx=[6,3]) #disable the detect button for now
        #self.detectButton.grid(row=2, column=2)
        self.editButton = Button(self.frame, text="Edit", font=('Arial 10'))
        self.editButton.pack(side="left", padx=3)
        #self.editButton.grid(row=2, column=3)
        
    def toggleEdit(self): #flip the editMode state
        if(self.editMode):
            self.stopEdit()
        else:
            self.startEdit()
    def startEdit(self):
        self.editMode = True
        self.editButton.configure(bg=self.editColor)
        self.frame.configure(highlightthickness=5)
    def stopEdit(self):
        self.editMode = False
        self.editButton.configure(bg="#F0F0F0") #default color
        self.frame.configure(highlightthickness=0)
        
    def changeColor(self, color): #change the primary color of the frame
        self.frame.configure(bg=color)
        self.typeLabel.configure(bg=color)
        self.groupLabel.configure(bg=color)
        self.nameLabel.configure(bg=color)

class StrokeFrame():
    def __init__(self, app):
        self.white = app.offWhite #used for stroke item
        #code for the stroke frame in the bottom left
        self.frame = Frame(app.bottomFrame, bg=app.lightBlue, width=420)
        self.frame.pack_propagate(0)
        self.frame.pack(side='left',  fill='y', padx=app.padding, pady=app.padding)
        #weird configuration needed for scrollsing, needs a canvas, and frame inside
        self.canvas = Canvas(self.frame, bg=app.lightBlue)
        self.scrollFrame = Frame(self.canvas, bg=app.lightBlue)
        self.scroll = Scrollbar(self.frame, orient="vertical", command=self.canvas.yview)
        self.canvas.configure(yscrollcommand=self.scroll.set)
        self.scroll.pack(side="right", fill='y')
        self.canvas.pack(side="left", fill="both", expand=1) 
        #configuration for stroke items
        self.strokeList = [] #list of stroke items
        self.strokeOptions = [*str("㇀㇁㇂㇃㇄㇅㇆㇇㇈㇉㇊㇋㇌㇍㇎㇏㇐㇑㇒㇓㇔㇕㇖㇗㇘㇙㇚㇛㇜㇝㇞㇟㇠㇡㇢㇣")] #list where each char is an entry
        self.groupOptions = range(1,15) #list where each char is an entry
        #content
        self.newButton = Button(self.scrollFrame, text="New Stroke", padx=40, font=('Arial 18'))
        self.newButton.pack(side='bottom', fill='y', padx=10, pady=10)
        #events
        self.canvas.bind("<Configure>", self.onFrameConfigure)
        self.canvas.bind("<Configure>", self.onFrameConfigure)
        self.scrollFrame.bind_all("<MouseWheel>", self.mouseScroll)
        
        self.isAdding = False #used to prevent errors when quickly adding strokes
    
    def addNewStrokeItem(self, delete=True): #delete chooses if the stroke is deleteable or not
        newItem = StrokeItem(self, len(self.strokeList)+1, self.white, allowDelete=delete) #StrokeFrame, strokeNum, backgroundColor
        self.strokeList.append(newItem)
        self.frame.update() #forces scrollbar to update
        self.onFrameConfigure(0) #redo the frame since we've added contents
        self.canvas.yview_moveto(1) #scroll to the bottom of the frame
        
    def onFrameConfigure(self, event): #note event isn't
        self.canvas.configure(scrollregion = self.canvas.bbox("all"))
        wrapFrame = self.canvas.create_window((0,0), window=self.scrollFrame, anchor="nw")
        self.canvas.itemconfigure(wrapFrame, width=self.canvas.winfo_width())
    
    def mouseScroll(self, event):
        #thanks: https://stackoverflow.com/questions/17355902/tkinter-binding-mousewheel-to-scrollbar
        self.canvas.yview_scroll(1,"units") if(event.delta < 0) else self.canvas.yview_scroll(-1,"units") # scroll up else scroll down
        
    def clearStrokeItems(self): #used to reset the stroke item list
        for ind in range(len(self.strokeList)):
            self.strokeList[ind].frame.destroy() #remove this stroke item from the ui
        self.strokeList.clear() #delete the actual objects
        self.frame.update() #forces scrollbar to update
        self.onFrameConfigure(0) #allows scrollbar to be updated
        
    def adjustColoring(self): #adjust colors for all the stroke boxes
        count = len(self.strokeList)
        colors = getColors(count)
        for i in range(count):
            self.strokeList[i].changeColor(colors[i])
        
class PictureFrame():
    def __init__(self, app):
        self.frame = Frame(app.bottomFrame, bg=app.mainBlue)
        self.frame.pack(side='right', fill='both', expand=True, padx = app.padding,  pady = app.padding)
        #content
        self.canvasSize=100 #not correct initial size, but the resize event will fix this on app startup
        self.canvas = Canvas(self.frame, bg="white", width=self.canvasSize, height=self.canvasSize) #our drawing board
        self.canvas.pack(side="top")
        #events
        self.frame.bind('<Configure>', self.resize)
        
        self.fitCopy = None #used for redrawing on resizing, note this is a shallow copy
        self.charImage = None #background image object
        self.boxes = [] #stroke bounding boxes
        self.strokes = [] #stroke curve lines, each element is a list of the segments for that stroke
        self.edit = -1 #-1 means no stroke has edit priority, otherwise that index matches the stroke number
        self.controlPoints = [] #plotting control points
        self.editControlPoints = False
        self.eval = np.linspace(0.0, 1.0, 25) #used for plotting bezier curves, parametric values
        
    def resize(self,event):
        oldSize = self.canvasSize
        self.canvasSize = min(event.width, event.height)-10
        self.canvas.config(width=self.canvasSize, height=self.canvasSize)
        if(oldSize != self.canvasSize): #don't redraw unless new size
            self.setFromFits(self.fitCopy, setCopy=False)
        
    def setFromFits(self, fitData, setCopy=True): #set copy is if we update our local copy of the fit
        if(fitData is None): #this event was called before data was set, ignore
            return
        if(setCopy):
            self.fitCopy = fitData
        self.boxes = [] #reset since we're redrawing
        self.controlPoints = [] #when we're editing a stroke we can edit control points as well
        self.strokes = []
        self.colors = [] #hex code of all the box colors
        self.canvas.delete('all') #remove all fits
        #load and set the char image
        arrayImg = 200 + 55*(1-fi.renderChar(fitData.character, size=1000, pad=.1)) #white maps to 255, black maps to 200, pad .1 is typical
        arrayImg = cv2.resize(arrayImg, dsize=(self.canvasSize, self.canvasSize), interpolation=cv2.INTER_NEAREST)
        self.img = ImageTk.PhotoImage(Image.fromarray(arrayImg)) #has to be saved to avoid being garbage collected
        self.charImage = self.canvas.create_image(0,0, image=self.img, anchor="nw")
        
        mappedFits = sf.mapFits(fitData.fits, fitData.size, [self.canvasSize, self.canvasSize], 0) #padding should be the same on both
        self.colors = getColors(len(fitData.fits), v=.8)
        for i in range(len(fitData.fits)):
            fit = mappedFits[i]
            self.boxes.append(self.canvas.create_rectangle(fit.x[0],fit.y[0],fit.x[1],fit.y[1], outline=self.colors[i], width=1, dash=(3,3)))
            
            curveLines = []
            stroke = sf.mapStroke(fitData.fits[i].data.arial, fit.x, fit.y)
            for seg in stroke:
                nodes = np.array(seg).transpose()
                curve = bezier.Curve(nodes, degree=len(seg)-1)
                curvePoints = curve.evaluate_multi(self.eval).transpose().tolist() #[x_value list, y_value list]
                curveLines.append(self.canvas.create_line(curvePoints, fill='#000000', width=3))
            self.strokes.append(curveLines)
        if(self.edit != -1): #adjust the edited stroke if need be
            self.editStroke(self.edit, True)
            
    def editStroke(self, ind, editMode):
        for point in self.controlPoints:
            self.canvas.delete(point) #delete previous points
        points = []
        if(editMode):
            self.canvas.itemconfig(self.boxes[ind], dash=(), width=2)
            #also plot the control points
            if(self.editControlPoints):
                mappedFit = sf.mapFits([self.fitCopy.fits[ind]], self.fitCopy.size, [self.canvasSize, self.canvasSize], 0)[0] #padding should be the same on both
                mappedStroke = sf.mapStroke(mappedFit.data.arial, mappedFit.x, mappedFit.y)
                for seg in mappedStroke:
                    for point in seg:
                        self.controlPoints.append( self.canvas.create_oval(point[0]-5,point[1]-5,point[0]+5,point[1]+5, width=0, fill=self.colors[ind]) )
                        self.canvas.tag_raise(self.controlPoints[-1]) #draw on top of other objects
        else:
            self.canvas.itemconfig(self.boxes[ind], dash=(3,3), width=1)
            
    def redrawStroke(self, ind, editMode):
        self.canvas.delete(self.boxes[ind]) #delete box and curve segments
        for curveId in self.strokes[ind]:
            self.canvas.delete(curveId)
        
        fit = sf.mapFits([self.fitCopy.fits[ind]], self.fitCopy.size, [self.canvasSize, self.canvasSize], 0)[0] #map only this fit for the bounding box
        self.boxes[ind] = self.canvas.create_rectangle(fit.x[0],fit.y[0],fit.x[1],fit.y[1], outline=self.colors[ind], width=1, dash=(3,3))
        self.editStroke(ind, editMode)
        
        curveLines = []
        stroke = sf.mapStroke(self.fitCopy.fits[ind].data.arial, fit.x, fit.y)
        for seg in stroke:
            nodes = np.array(seg).transpose()
            curve = bezier.Curve(nodes, degree=len(seg)-1)
            curvePoints = curve.evaluate_multi(self.eval).transpose().tolist() #[x_value list, y_value list]
            curveLines.append(self.canvas.create_line(curvePoints, fill='#000000', width=3))
        self.strokes[ind] = curveLines
            
        
class OptionsFrame():
    def __init__(self, app):
        self.frame = Frame(app.topFrame, bg=app.mainBlue)
        self.frame.pack(side = 'right',  fill = 'both',  padx = app.padding,  pady = app.padding,  expand=True)
        #content
        self.autoButton = Button(self.frame, text="Auto-Gen", font=('Arial 14'))
        self.autoButton.pack(side='left', padx=20, pady=10)
        self.loadButton = Button(self.frame, text="Load", font=('Arial 14'))
        self.loadButton.pack(side='left', padx=20, pady=10)
        
        self.controlPointsVar = IntVar()
        self.controlPointsButton = Checkbutton(self.frame, text="Edit Control Points", font=('Arial 14'), variable=self.controlPointsVar, onvalue=1, offvalue=0)
        self.controlPointsButton.pack(side='left', padx=20, pady=10)
                                      
        self.exportButton = Button(self.frame, text="Save & Export", font=('Arial 14'))
        self.exportButton.pack(side='right', padx=20, pady=10)
        self.saveButton = Button(self.frame, text="Save", font=('Arial 14'))
        self.saveButton.pack(side='right', padx=20, pady=10)
        #events

class App:
    def run(self):
        self.root.mainloop() #run the app
    
    def __init__(self, width = 1200, height = 850, padding=10):
        #setup vars for colors and config
        self.width = width
        self.height = height
        self.padding = padding
        self.mainBlue = "#5A837B"
        self.lightBlue = "#B3CCC8"
        self.offWhite = "#CFE7E2"
        
        #setup initial app window
        self.root = Tk()
        self.root.geometry( str(width) + "x" + str(height) )
        self.root.configure(background=self.mainBlue)
        #top frame for options and character
        self.topFrame = Frame(self.root, bg=self.mainBlue)
        self.topFrame.pack(side='top', fill="both")
        #bottom frame for stroke editor and picture frame
        self.bottomFrame = Frame(self.root, bg=self.mainBlue)
        self.bottomFrame.pack(side='bottom', fill="both", expand=True)

        self.cf = CharFrame(self) #init the top left character frame
        self.of = OptionsFrame(self) #init the top right options frame
        self.sf = StrokeFrame(self) #init the bottom left stroke frame
        self.pf =  PictureFrame(self) #init the bottom right picture frame
        
        self.me = MiddleEvents(self) #register all the important events
        
#registers all events needed to communicate with the middle layer e.g. rendering images
class MiddleEvents:
    def __init__(self, app):
        self.app = app
        
        self.fitData = FitData()
        self.fitData.set("add char", [], [1000,1000]) #init with default vals, character, fits, size
        
        self.strokeDict = sd.loadStrokeDict()
        self.strokeNames = []
        for strokeType in self.strokeDict.values():
            self.strokeNames.append(strokeType.symbol + " " + strokeType.name) #e.g. D1 Vert |
        self.app.sf.strokeOptions = self.strokeNames #update drop down list for strokes
        
        self.app.of.autoButton.configure(command = self.autoClick)
        self.app.of.loadButton.configure(command = self.loadClick)
        self.app.of.saveButton.configure(command = self.saveClick)
        self.app.of.exportButton.configure(command = self.exportClick)
        self.app.of.exportButton.configure(command = self.exportClick)
        self.app.of.controlPointsVar.trace("w", self.editControlChange)
        
        self.app.pf.canvas.bind_all("<KeyPress>", self.eventKeyPress)
        self.app.pf.canvas.bind("<Button-1>", self.canvasClick)
        self.app.pf.canvas.bind("<ButtonRelease-1>", self.canvasClickRelease) #not currently used
        self.app.root.bind("<Return>", self.newStrokeKeyPress)
        if(platform.system() == "Windows"):
            self.app.pf.canvas.bind("<Button-3>", self.canvasRightClick)
        else: #assume mac
            self.app.pf.canvas.bind("<Button-2>", self.canvasRightClick)
        self.app.pf.canvas.bind("<B1-Motion>", self.canvasDrag)
        
        self.app.cf.code.trace("w", self.codeChange)
        self.app.cf.char.trace("w", self.charChange)
        
        self.app.sf.newButton.configure(command = self.newStrokeAdded)
        
        self.codeUpdated = False #determines whether to use text from the code or char box to determine what character we're using.
        
        self.xEdge = -1 # -1: none, 0: left, 1:right
        self.yEdge = -1 # -1: none, 0: up, 1: down
        self.draggedPoints = [] #a list of lists, elemenst like [segIndex, pointIndex], indicates which points are currently being dragged.
        
        self.centerMove = None #move the whole stroke at once, None, otherwise, (clickX,clickY,origXRange, origYRange)
    
    def newStrokeKeyPress(self, event):
        self.newStrokeAdded()
        
    def editControlChange(self, *args):
        self.app.pf.editControlPoints = (self.app.of.controlPointsVar.get() == 1)
        
        editStroke = self.app.pf.edit
        if(editStroke != -1): #redraw stroke if needed
            self.app.pf.editStroke(editStroke, editStroke != -1)
    def eventKeyPress(self, event): #handles options for various types of key presses
        if(event.char == ' '): #space character, toggle control point editing
            self.app.of.controlPointsButton.toggle() #flip the box
        elif(event.keysym == "Delete"): #delete key
            if(self.app.pf.edit != -1): #a stroke is being edited
                self.strokeDelete(self.app.pf.edit) #delete the current edited stroke
    
    def codeChange(self, *args): #e.g. 45E2
        self.codeUpdated = True
        text = self.app.cf.code.get()
        filtered = "".join(re.findall("[a-f,A-F,\d]", text)) #only characters related to unicode are allowed
        if(len(filtered) > 6): #too long
            filtered = filtered[:6]
        self.app.cf.code.set(filtered)
        
    def charChange(self, *args): #character entered, one char max
        self.codeUpdated = False
        text = self.app.cf.char.get()
        if(len(text) > 1): #only one character is allowed
            self.app.cf.char.set(text[0])
        
    def getChar(self): #get the desired character (either from the code or char box), updates other to match
        if(self.codeUpdated):
            text = self.app.cf.codeEntry.get()
            character =  chr(int(text, 16)) #char from the unicode num in hex
            self.app.cf.char.set(character)
        else: #char box
            character = self.app.cf.charEntry.get()
            self.app.cf.code.set( hex(ord(character))[2:] )
        return character

    def setFitData(self, tempData):
        self.app.pf.edit = -1 #reset the edited stroke to none
        self.app.sf.clearStrokeItems() #reset the stroke item list
        char = self.getChar()
        if(tempData is None): #just in case
            print(f"{char} fit data not found!")
            self.fitData = FitData()
            self.fitData.set(char, [], [1000,1000]) #set a blank fits            
            self.app.pf.setFromFits(self.fitData) #empty data
            return
        self.fitData = tempData #set, since data loaded properly
        for i in range(len(self.fitData.fits)):
            fit = self.fitData.fits[i]
            self.app.sf.addNewStrokeItem()
            self.app.sf.strokeList[i]
            #change the properties of item to match the fit
            strokeType = fit.data
            self.app.sf.strokeList[i].strokeChoice.set(strokeType.symbol + " " + strokeType.name)
            self.app.sf.strokeList[i].groupChoice.set(fit.group)
            #bind events
            self.app.sf.strokeList[i].strokeEvent = self.app.sf.strokeList[i].strokeChoice.trace("w", lambda *args, ind=i, me=self: me.strokeOptionChange(ind))
            self.app.sf.strokeList[i].groupEvent = self.app.sf.strokeList[i].groupChoice.trace("w", lambda *args, ind=i, me=self: me.strokeGroupChange(ind))
            self.app.sf.strokeList[i].deleteButton.configure(command = lambda ind=i, me=self: me.strokeDelete(ind))
            self.app.sf.strokeList[i].detectButton.configure(command = lambda ind=i, me=self: me.strokeDetect(ind)) 
            self.app.sf.strokeList[i].editButton.configure(command = lambda ind=i, me=self: me.strokeEdit(ind))
        #set canvas objects and update
        self.app.pf.setFromFits(self.fitData)
        self.app.sf.adjustColoring()
    
    def autoClick(self):
        char = self.getChar()
        tempData = bp.fitChar(char, size=1000)
        self.setFitData(tempData)

        if(not fd.fileExists(char)): #save auto-gen result if no previous data exists for this char
            fd.saveFits(self.fitData)
        
    def loadClick(self):
        char = self.getChar()
        if(fd.fileExists(char)): #use existing saved data
            self.setFitData( fd.loadFits(char) )
        else: #use autogen since no data is saved
            self.autoClick() #auto-gen data since no data is saved
        
        
    def saveClick(self):
        fd.saveFits(self.fitData)
        #convert canvas objects and save
    def exportClick(self):
        #save & and then export
        self.saveClick()
        XMLMaker.save(self.fitData.character) #export to an xml
        
    
    #canvas events ----
    def canvasClick(self, event):
        if(self.app.pf.edit != -1): #a certain stroke is being edited
            #map x and y to proper position
            x = sf.mapCoord(event.x, self.app.pf.canvasSize, self.fitData.size[0], 0) #0 is the padding
            y = sf.mapCoord(event.y, self.app.pf.canvasSize, self.fitData.size[1], 0)
            fit = self.fitData.fits[self.app.pf.edit]
            
            self.xEdge = -1
            self.yEdge = -1
            self.centerMove = None
            self.draggedPoints = [] #reset
            
            closestPoint = 10e10 #arbitrary large value
            if(self.app.pf.editControlPoints): #we are moving control points, note we can drag multiple control points at a time if they are on top of eachother
                minDist = 20 #how far the mouse can be from a point, manhattan distance                
                stroke = sf.mapStroke(fit.data.arial, fit.x, fit.y)
                for segInd in range(len(stroke)):
                    for pointInd in range(len(stroke[segInd])):
                        point = stroke[segInd][pointInd]
                        dist = abs(x-point[0]) + abs(y-point[1])
                        closestPoint = min(dist, closestPoint)
                        if(dist <= minDist): #close to the point
                            self.draggedPoints.append([segInd, pointInd])
            
            if(not self.app.pf.editControlPoints or closestPoint > 50): #we are moving the bounding box, either because control edit is off, or we clicked far from any points           
                #figure out which edge is closest
                leftDist  = abs(x-fit.x[0]) + betweenDist(y, fit.y[0], fit.y[1])
                rightDist = abs(x-fit.x[1]) + betweenDist(y, fit.y[0], fit.y[1])
                upDist    = abs(y-fit.y[0]) + betweenDist(x, fit.x[0], fit.x[1])
                downDist  = abs(y-fit.y[1]) + betweenDist(x, fit.x[0], fit.x[1])

                minDist = 25 #furthest mouse can be away to count as a valid click

                currDist = 10e10
                if(leftDist < minDist):
                    currDist = leftDist
                    self.xEdge = 0
                if(rightDist < currDist and rightDist < minDist):
                    currDist = rightDist
                    self.xEdge = 1
                currDist = 10e10
                if(upDist < currDist and upDist < minDist):
                    currDist = upDist
                    self.yEdge = 0
                if(downDist < currDist and downDist < minDist):
                    self.yEdge = 1 #no need to set min dist, this is the last one

                centerDist = 25 #must be at least this many pixels from edge to be considered for a center drag
                #if we're not clicking the edges, move the whole stroke if we're clicking inside it
                if(self.xEdge == -1 and self.yEdge == -1 and betweenDist(x, fit.x[0], fit.x[1])==0 and betweenDist(y, fit.y[0], fit.y[1])==0):
                    marginDist = min(leftDist,rightDist,upDist,downDist)
                    if(marginDist > centerDist): #we're clicking in the box and aren't near any edges
                        self.centerMove = (x,y, fit.x, fit.y)
            
    def canvasDrag(self, event): #"<B1-Motion>"
        if(self.app.pf.edit != -1): #a certain stroke is being edited and mouse clicked on an edge
            i = self.app.pf.edit #stroke editing index
            x = sf.mapCoord(event.x, self.app.pf.canvasSize, self.fitData.size[0], 0) #0 is the padding
            y = sf.mapCoord(event.y, self.app.pf.canvasSize, self.fitData.size[1], 0)
            
            if(self.app.pf.editControlPoints): #we are moving control points, note we can drag multiple control points at a time if they are on top of eachother
                fit = self.fitData.fits[self.app.pf.edit]
                xMapped = (x-fit.x[0])/abs(fit.x[1]-fit.x[0]) #map to a normalized 0-1 box
                yMapped = (y-fit.y[0])/abs(fit.y[1]-fit.y[0])                
                if(fit.x[0] > fit.x[1]): #special case when lower/upper bounds are out of order
                    xMapped = xMapped*-1
                if(fit.y[0] > fit.y[1]):
                    yMapped = yMapped*-1
                
                for pointLoc in self.draggedPoints:
                    self.fitData.fits[self.app.pf.edit].data.arial[pointLoc[0]][pointLoc[1]] = [xMapped,yMapped]
                
            #move the bounding box if needed
            if(self.xEdge == 0): #left
                self.fitData.fits[i].x[0] = x
            elif(self.xEdge == 1): #right
                self.fitData.fits[i].x[1] = x

            if(self.yEdge == 0): #up
                self.fitData.fits[i].y[0] = y
            elif(self.yEdge == 1): #right: #down
                self.fitData.fits[i].y[1] = y

            if(self.centerMove != None): #make x,y the center of the stroke
                self.fitData.fits[i].x = self.centerMove[2] + (x - self.centerMove[0]) #offset from original left click pos
                self.fitData.fits[i].y = self.centerMove[3] + (y - self.centerMove[1])
                
            self.app.pf.redrawStroke(i, True) #second arg is edit mode
            
    def canvasClickRelease(self, event):
        #just reset the move type properties, isn't strictly necessary, but avoids stale vars
        self.xEdge = -1 # -1: none, 0: left, 1:right
        self.yEdge = -1 # -1: none, 0: up, 1: down
        self.centerMove = None #move the whole stroke at once
    
    def canvasRightClick(self, event): #toggle edit off the nearest stroke, must be inside the bounding box, priority to the earier strokes.
        x = sf.mapCoord(event.x, self.app.pf.canvasSize, self.fitData.size[0], 0) #0 is the padding
        y = sf.mapCoord(event.y, self.app.pf.canvasSize, self.fitData.size[1], 0)
        
        bestDist = 10e10 #arbitrary large dist
        bestStroke = -1
        for i in range(len(self.fitData.fits)):
            fit = self.fitData.fits[i]
            #don't consider if mouse isn't in the bounding box
            if(fit.x[0] <= fit.x[1] and (x < fit.x[0] or fit.x[1] < x)):
                continue
            if(fit.x[0] >= fit.x[1] and (x < fit.x[1] or fit.x[0] < x)): #min/max are swapped
                continue
            if(fit.y[0] <= fit.y[1] and (y < fit.y[0] or fit.y[1] < y)):
                continue
            if(fit.y[0] >= fit.y[1] and (y < fit.y[1] or fit.y[0] < y)): #min/max are swapped
                continue
            dist = abs(x - (fit.x[0]+fit.x[1])/2) + abs(y - (fit.y[0]+fit.y[1])/2) #distance from center of bounding box
            if(dist < bestDist):
                bestDist = dist
                bestStroke = i       
        
        if(bestStroke != -1): #edit this stroke
            self.strokeEdit(bestStroke) #clicking on canvas stroke is essentially the same as clicking on edit button
    
    def newStrokeAdded(self): #bind events for the newly added stroke item
        if(len(self.fitData.fits) == 0): #first stroke added, load the char
            self.fitData.character = self.getChar()
            self.app.pf.setFromFits(self.fitData) #set the image
        
        if(self.app.sf.isAdding):
            return
        self.app.sf.isAdding = True #essentially using this is a mutex lock
        self.app.sf.addNewStrokeItem() #add new stroke item
        
        #bind events
        index = len(self.app.sf.strokeList)-1
        if(index != 0): #configure the group option so it is the same group as the previous stroke
            self.app.sf.strokeList[index].groupChoice.set(self.app.sf.strokeList[index-1].groupChoice.get())
        self.app.sf.strokeList[index].strokeEvent = self.app.sf.strokeList[index].strokeChoice.trace("w", lambda *args, ind=index, me=self: me.strokeOptionChange(ind))
        self.app.sf.strokeList[index].groupEvent = self.app.sf.strokeList[index].groupChoice.trace("w", lambda *args, ind=index, me=self: me.strokeGroupChange(ind))
        self.app.sf.strokeList[index].deleteButton.configure(command = lambda ind=index, me=self: me.strokeDelete(ind))
        self.app.sf.strokeList[index].detectButton.configure(command = lambda ind=index, me=self: me.strokeDetect(ind)) 
        self.app.sf.strokeList[index].editButton.configure(command = lambda ind=index, me=self: me.strokeEdit(ind))
        
        strokeName = self.app.sf.strokeList[index].strokeChoice.get()[2:] #skip symbol and space character
        windowSize = self.fitData.size #2d array, [width, height] (but they should be the same)
        xRange = [int(windowSize[0]*.333), int(windowSize[0]*.666)] #position in the midel taking up one 3rd the window size
        yRange = [int(windowSize[1]*.333), int(windowSize[1]*.666)]
        self.fitData.fits.append(sf.Fit(strokeName, self.strokeDict[strokeName], xRange, yRange))
        
        self.app.pf.setFromFits(self.fitData) #redraw canvas
        self.app.sf.adjustColoring() #recolor all the strokes evenly with hsv
        
        self.app.sf.isAdding = False #remove the lock
    
    #stroke events, more difficult to work with
    def strokeOptionChange(self, ind):
        strokeName = self.app.sf.strokeList[ind].strokeChoice.get()[2:] #skip the symbol and space characters, just get the name
        self.fitData.fits[ind] = sf.Fit(strokeName, self.strokeDict[strokeName], self.fitData.fits[ind].x, self.fitData.fits[ind].y)
        self.app.pf.redrawStroke(ind, self.app.pf.edit == ind) #second arg is edit mode
        
    def strokeGroupChange(self, ind): #this probably shouldn't do anything, right?
        self.fitData.fits[ind].group = int(self.app.sf.strokeList[ind].groupChoice.get())
        
    def strokeDelete(self, ind):
        if(self.app.pf.edit == ind): #to prevent errors stop editing before deleting
            self.app.pf.edit = -1
        self.app.sf.strokeList[ind].frame.destroy() #remove this stroke item from the ui
        del self.app.sf.strokeList[ind]
        del self.fitData.fits[ind]
        
        #all strokes after the one we deleted are now incorrect, adjust them
        for i in range(ind, len(self.app.sf.strokeList)):
            #unbind events
            self.app.sf.strokeList[i].strokeChoice.trace_remove("write", self.app.sf.strokeList[i].strokeEvent)
            self.app.sf.strokeList[i].groupChoice.trace_remove("write", self.app.sf.strokeList[i].groupEvent)
            #button can just be reconfigured, no unbinding required
            
            #now rebind events
            self.app.sf.strokeList[i].strokeEvent = self.app.sf.strokeList[i].strokeChoice.trace("w", lambda *args, ind=i, me=self: me.strokeOptionChange(ind))
            self.app.sf.strokeList[i].groupEvent = self.app.sf.strokeList[i].groupChoice.trace("w", lambda *args, ind=i, me=self: me.strokeGroupChange(ind))
            self.app.sf.strokeList[i].deleteButton.configure(command = lambda ind=i, me=self: me.strokeDelete(ind))
            self.app.sf.strokeList[i].detectButton.configure(command = lambda ind=i, me=self: me.strokeDetect(ind)) 
            self.app.sf.strokeList[i].editButton.configure(command = lambda ind=i, me=self: me.strokeEdit(ind))
            
            #change the title
            self.app.sf.strokeList[i].nameLabel.configure(text = f"#{i+1}")
        
        self.app.pf.setFromFits(self.fitData)
        self.app.sf.adjustColoring() #reset background colors
        
        self.app.sf.frame.update() #forces scrollbar to update
        self.app.sf.onFrameConfigure(0) #allows scrollbar to be updated
        
    def strokeDetect(self, ind):
        print("detect stroke", ind)
    def strokeEdit(self, ind):
        if(not self.app.sf.strokeList[ind].editMode): #change to editing
            #disable edit on all other buttons
            for i in range(len(self.app.sf.strokeList)):
                self.app.sf.strokeList[i].stopEdit()
                self.app.pf.editStroke(i, False)
            self.app.sf.strokeList[ind].startEdit()
            self.app.pf.editStroke(ind, True)
            self.app.pf.edit = ind
        else: #turn off editing.
            self.app.sf.strokeList[ind].stopEdit()
            self.app.pf.editStroke(ind, False)
            self.app.pf.edit = -1 #no stroke is being edited

    #still need to write events for:
    #auto-gen
    #save
    #save & export
    #add hide bounding box key? maybe alt?
    
app = App()
app.run()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\daniel\AppData\Local\Programs\Python\Python38\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\daniel\AppData\Local\Temp\ipykernel_41432\3008824673.py", line 441, in autoClick
    tempData = bp.fitChar(char, size=1000)
  File "C:\Users\daniel\Documents\StylusArchetypeConverter\Code\BatchProcess.py", line 33, in fitChar
    fitData = sm.remapFits(character, fitData=fitData)
  File "C:\Users\daniel\Documents\StylusArchetypeConverter\Code\Anchors.py", line 432, in remapFits
    if(not self.isValidMap(origFit, newFit, img)):
  File "C:\Users\daniel\Documents\StylusArchetypeConverter\Code\Anchors.py", line 302, in isValidMap
    if(img[y,x] <= 0):
IndexError: index 1016 is out of bounds for axis 1 with size 1000


In [44]:
import platform
print(platform.system())
data = (0,7,[0,2])
print(data)
print(data[0])
print(data[2])

Windows
(0, 7, [0, 2])
0
[0, 2]


In [7]:
x = np.zeros((5,5))
print(np.shape(x))
size1 = np.shape(x)
print(size1[0], size1[1])

(5, 5)
5 5
